## Eli Dow

## HW05 Code


You will complete the following notebook, as described in the PDF for Homework 05 (included in the download with the starter code).  You will submit:
1. This notebook file, along with your COLLABORATORS.txt file and the two tree images (PDFs generated using `graphviz` within the code), to the Gradescope link for code.
2. A PDF of this notebook and all of its output, once it is completed, to the Gradescope link for the PDF.


Please report any questions to the [class Piazza page](https://piazza.com/tufts/spring2021/comp135).

### Import required libraries.

In [1]:
import numpy as np
import pandas as pd

import sklearn.tree
import graphviz

# for log function
import math

## Decision Trees

You should start by computing the two heuristic values for the toy data described in the assignment handout. You should then load the two versions of the abalone data, compute the two heuristic values on features (for the simplified data), and then build decision trees for each set of data.

### 1 Compute both heuristics for toy data.

### Description of Data Sets

The data sets represent the best toys for kids, and are based on the decision tree diagrams in the spec. The data-set consists of eight entries (children) and whether they would prefer an action-figure (1) or doll (0). Each entry has two features: is_male and likes_superheroes. is_male represents whether the child is male (1) or female (0), and likes_superheroes represents whether the child likes superheroes (1) or not (0).

#### Load Toy Data Sets

In [2]:
x_toydata = np.loadtxt('data_toys/x_data.csv', skiprows=1, delimiter=',')
y_toydata = np.loadtxt('data_toys/y_data.csv', skiprows=1, delimiter=',')

#### (a) Compute the counting-based heuristic, and order the features by it.

In [3]:
def compute_count_heuristic(x_data, y_data, outcomes, empty_h_values):
    ''' Computes the counting heuristic values for each feature of x_data
    
    Args
    ----
    x_data : 2D list of binary values
        Each element in x_data represents a list of each data points features
        Each feature is either a 1 or 0
    y_data : 1D list of integers
        Each element in y_data is an integer representing a particular classification
    outcomes : integer value
        Represents the number of possible classification outcomes
    empty_h_values : list of tuples
        Each tuple contains a feature name and its initial counting heuristic value (0)
        
    Returns
    -------
    heuristic_values: list of tuples
        Each tuple contains a feature name and its computed counting heuristic value (0)
    '''
    
    # intialize heuristic values and total data
    heuristic_values = empty_h_values
    total_data = len(y_data)

    # for each attribute
    for feature in range(len(heuristic_values)):

        # initialize positive features subset and data corresponding to them
        true_feat = []
        true_outcomes = [0] * outcomes
        true_majority = 0
        
        # initialize false features subset and data corresponding to them
        false_feat = []
        false_outcomes = [0] * outcomes
        false_majority = 0

        # initialize variable for y_data based on subset majority
        y_data_majority = []

        # divide data set according to possible values of that attribute
        # keep track of number of classifications for each subset
        for dp in range(total_data):
            if(x_data[dp][feature] == 1):
                true_feat.append(dp)
                true_outcomes[int(y_data[dp])] += 1
            else:
                false_feat.append(dp)
                false_outcomes[int(y_data[dp])] += 1

        # find majority in each subset
        for i in range(1, len(true_outcomes)):
            if (max(true_outcomes) == true_outcomes[i]):
                true_majority = i
            if (max(false_outcomes) == false_outcomes[i]):
                false_majority = i

        # assign all data points the majority value in its subset
        for dp in range(total_data):
            if len(true_feat) > 0 and dp == true_feat[0]:
                y_data_majority.append(true_majority)
                true_feat.pop(0)
            elif len(false_feat) > 0 and dp == false_feat[0]:
                y_data_majority.append(false_majority)
                false_feat.pop(0)

        # count total correct and store it
        num_correct = 0
        for dp in range(total_data):
            if y_data_majority[dp] == y_data[dp]:
                num_correct += 1
        heuristic_values[feature] = tuple([heuristic_values[feature][0], num_correct])

    # sort by heuristic value, descending order
    heuristic_values.sort(key=lambda a: a[1])
    heuristic_values.reverse()
    
    return heuristic_values


# compute counting heuristic for toy data
toy_h_values = compute_count_heuristic(x_toydata, y_toydata, 2, [('is_male', 0.0), ('likes_superheroes', 0.0)])

# print out toy features based upon counting heuristic 
for feature in range(len(toy_h_values)):
    feature_name = toy_h_values[feature][0]
    heuristic_v = toy_h_values[feature][1]
    decimal_h = round(heuristic_v / len(y_toydata), 3)
    print(str(feature_name) + ": " + str(heuristic_v) + "/" + str(len(y_toydata)) + " (" + str(decimal_h) + ")")    

likes_superheroes: 6/8 (0.75)
is_male: 6/8 (0.75)


#### (b) Compute the information-theoretic heuristic, and order the features by it.

In [4]:
def compute_info_heuristic(x_data, y_data, outcomes, empty_h_values):
    ''' Computes the information gain values for each feature of x_data
    
    Args
    ----
    x_data : 2D list of binary values
        Each element in x_data represents a list of each data points features
        Each feature is either a 1 or 0
    y_data : 1D list of integers
        Each element in y_data is an integer representing a particular classification
    outcomes : integer value
        Represents the number of possible classification outcomes
    empty_h_values : list of tuples
        Each tuple contains a feature name and its initial information gain value (0)
        
    Returns
    -------
    info_gains: list of tuples
        Each tuple contains a feature name and its computed information gain value (0)
    '''
    
    # intialize information gains and total data
    info_gains = empty_h_values
    total_data = len(y_data)

    # for each attribute
    for feature in range(len(info_gains)):

        # initialize variables for y_data
        results = [0] * outcomes
        frac_outcomes = [0] * outcomes
        
        # initialize positive features subset and data corresponding to them
        true_feat = 0
        true_outcomes = [0] * outcomes
        true_frac = [0] * outcomes
        
        # initialize false features subset and data corresponding to them
        false_feat = 0
        false_outcomes = [0] * outcomes
        false_frac = [0] * outcomes

        # count each classification and store in results
        for i in range(total_data):
            results[int(y_data[i])] += 1
    
        # find fraction of each classifion over total data
        for i in range(len(results)):
            frac_outcomes[i] = results[i] / total_data
        
        # calculate h_example
        h_example = 0
        for i in range(len(frac_outcomes)):
            h_example += (frac_outcomes[i] * math.log(frac_outcomes[i], 2))
        
        h_example *= -1

        # count each subset according to possible values of that attribute
        # keep track of number of classifications for each subset
        for i in range(total_data):
            if x_data[i][feature] == 1:
                true_feat += 1
                true_outcomes[int(y_data[i])] += 1
            else:
                false_feat += 1
                false_outcomes[int(y_data[i])] += 1
        
        # find fraction of each classifion over total data 
        for i in range(len(true_outcomes)):
            true_frac[i] = true_outcomes[i] / true_feat
            false_frac[i] = false_outcomes[i] / false_feat
        
        # initialize variables for r_cost
        r_cost = 0
        true_h_ex = 0
        false_h_ex = 0
        
        # Solve for r_cost
        if(0 not in true_frac):
            for i in range(len(true_outcomes)):
                true_h_ex += (true_frac[i] * math.log(true_frac[i], 2))
        
        true_h_ex *= -1

        if(0 not in false_frac):
            for i in range(len(false_outcomes)):
                false_h_ex += (false_frac[i] * math.log(false_frac[i], 2))
        
        false_h_ex *= -1

        r_cost = (true_h_ex * (true_feat / total_data)) + (false_h_ex * (false_feat / total_data))

        # solve for information gain, set to info_gains
        gain = round(h_example - r_cost, 3)
        info_gains[feature] = tuple([info_gains[feature][0], gain])
        
    # sort by information gain, descending order
    info_gains.sort(key=lambda a: a[1])
    info_gains.reverse()
    return info_gains


# compute information gain for toy data
toy_info_gains = compute_info_heuristic(x_toydata, y_toydata, 2, [('is_male', 0.0), ('likes_superheroes', 0.0)])

# print out toy features based upon information gain
for feature in range(len(toy_info_gains)):
    print(str(toy_info_gains[feature][0]) + ": " + str(toy_info_gains[feature][1]))  

is_male: 0.311
likes_superheroes: 0.189


#### (c) Discussion of results.

Based on the results from part one, we can determine a few things when building a decision tree for these heuristics. 

First, we determined the counting heuristic values for each feature of the toy data. The counting heuristic value for both features was 3/4. This means that when we broke up the data set by each feature, then classified every data point by the majority classification of that subset, it was consistent with the ouput data 75% of the time. Thus, the counting heuristic doesn't tell us which feature we should start building the tree with from the root. In general, the counting heuristic isn't the best measure to determine important attributes for building decision trees. This method is helpful for determining the next attribute (good short-term), but it is a greedy algorithm that won't always get you the most efficient/smalled decision tree (bad long-term).

Second, we determined the information gain values for each feature of the toy data. The information gain value for is_male is 0.311, and the value for likes_superheroes is 0.189. In information theory, if we do not know whether an event has happenned or not, then learning facts is a gain in information. Intuitively, a good attribute choice when building a tree is one that gives us the most information about how output decisions are determined. This means that is_male is giving us more information about how output decisions are made, and would create a better decision tree if picked first. On the other hand, likes_superheroes doesn't give us as much information. Despite it having the same counting heuristic value, its information gain demonstrates that it would be a bad first choice for the decision tree long-term.

### 2 Compute both heuristics for simplified abalone data.

#### Load Abalone Data Sets

In [5]:
# simple data
sb_x_train = np.loadtxt('data_abalone/small_binary_x_train.csv', skiprows=1, delimiter=',')
sb_x_test = np.loadtxt('data_abalone/small_binary_x_test.csv', skiprows=1, delimiter=',')

threec_y_train = np.loadtxt('data_abalone/3class_y_train.csv', skiprows=1, delimiter=',')
threec_y_test = np.loadtxt('data_abalone/3class_y_test.csv', skiprows=1, delimiter=',')

# complex data
x_train = np.loadtxt('data_abalone/x_train.csv', skiprows=1, delimiter=',')
x_test = np.loadtxt('data_abalone/x_test.csv', skiprows=1, delimiter=',')

y_train = np.loadtxt('data_abalone/y_train.csv', skiprows=1, delimiter=',')
y_test = np.loadtxt('data_abalone/y_test.csv', skiprows=1, delimiter=',')

#### (a) Compute the counting-based heuristic, and order the features by it.

In [6]:
# initialize list of feature names and 0 intital heuristic values
empty_h_values_abalone = [('is_male', 0.0), ('length_mm', 0.0), ('diam_mm', 0.0), ('height_mm', 0.0)]

# compute counting heuristic for simple abalone data
sab_h_values = compute_count_heuristic(sb_x_train, threec_y_train, 3, empty_h_values_abalone)

# print out simple abalone features based upon counting heuristic 
for feature in range(len(sab_h_values)):
    feature_name = sab_h_values[feature][0]
    heuristic_v = sab_h_values[feature][1]
    decimal_h = round(heuristic_v / len(threec_y_train), 3)
    print(str(feature_name) + ": " + str(heuristic_v) + "/" + str(len(threec_y_train)) + " (" + str(decimal_h) + ")")
    

height_mm: 2316/3176 (0.729)
diam_mm: 2266/3176 (0.713)
length_mm: 2230/3176 (0.702)
is_male: 1864/3176 (0.587)


#### (b) Compute the information-theoretic heuristic, and order the features by it.

In [7]:
# initialize list of feature names and 0 intital information gain values
empty_h_values_abalone = [('is_male', 0.0), ('length_mm', 0.0), ('diam_mm', 0.0), ('height_mm', 0.0)]

# compute information gain for simple abalone data
sab_info_gains = compute_info_heuristic(sb_x_train, threec_y_train, 3, empty_h_values_abalone)

# print out simple abalone features based upon information gain
for feature in range(len(sab_info_gains)):
    print(str(sab_info_gains[feature][0]) + ": " + str(sab_info_gains[feature][1]))  

height_mm: 0.173
diam_mm: 0.15
length_mm: 0.135
is_male: 0.025


### 3 Generate decision trees for full- and restricted-feature data

#### (a) Print accuracy values and generate tree images.

In [8]:
# create/fit classifier for decision tree of simple abalone data
simple_clf = sklearn.tree.DecisionTreeClassifier(criterion='entropy')
simple_clf.fit(sb_x_train, threec_y_train)

# calculate/print accuraccies for simple abalone data
simple_train_score = simple_clf.score(sb_x_train, threec_y_train)
simple_test_score = simple_clf.score(sb_x_test, threec_y_test)

print("Accuracy on simple training data: " + str(round(simple_train_score, 3)))
print("Accuracy on simple testing data: " + str(round(simple_test_score, 3)))

# create/fit classifier for decision tree of regular abalone data
regular_clf = sklearn.tree.DecisionTreeClassifier(criterion='entropy')
regular_clf.fit(x_train, y_train)

# calculate/print accuraccies for regular abalone data
regular_train_score = regular_clf.score(x_train, y_train)
regular_test_score = regular_clf.score(x_test, y_test)

print("Accuracy on regular training data: " + str(round(regular_train_score, 3)))
print("Accuracy on regular testing data: " + str(round(regular_test_score, 3)))

# export simple data decision tree as pdf
simp_export = sklearn.tree.export_graphviz(simple_clf, out_file=None)
simp_tree = graphviz.Source(simp_export)
simp_tree.render('simple_tree')

# export regular data decision tree as pdf
regular_export = sklearn.tree.export_graphviz(regular_clf, out_file=None)
regular_tree = graphviz.Source(regular_export)
regular_tree.render('regular_tree')

Accuracy on simple training data: 0.733
Accuracy on simple testing data: 0.722
Accuracy on regular training data: 1.0
Accuracy on regular testing data: 0.206


'regular_tree.pdf'

#### (b) Discuss the results seen for the two trees

Based on the results from part three, we can determine a few things about the various accuracy-score values, the differences between the two trees, and their errors.

The accuracy for the simple training data is 73.3%, and the accuracy for the simple testing data is around the same value, slighlty less, at 72.2%. This tells me that the decision tree for simple data has sub-par yet consistent accurracy for all data. In other words, this model demonstrates underfitting. On the other hand, the accuracy for the regular training data was 100%, but the accuracy for the testing data was only 20%. This tells me that the decision tree for the regular data is perfect for the data it was trained with, but really bad for any other data. In other words, this model demonstrates overfitting.

Other than the fact that they are both binary trees (features are binary), the two trees differ greatly. The simple tree is relatively small, has a height of 4, is balanced, and has 16 leaves. The regular tree is very wide, has a height of at least 16, is far from a perfectly balanced tree, and has tons of leaves at various depth.

Looking at the leaves of the simple data decision tree, you can see the misclassifications in the value lists of each leaf. It appears that abalones with 2 rings (large) were misclassified as 1 ring abalones (medium), and even 0 ring ones (small). It also seems like lots of 0 ring abalones were misclassified as 1 ring.